In [1]:
!pip install tf_agents
!pip install yfinance
!pip install tensorflow-probability
!pip install tf-agents[reverb]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 KB 34.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.23.0-py3-none-any.whl size=697660 sha256=21157cb70dcc4fa0bd477fac018ada7ccbbdd21c22db1f4547a1e5e6001cff1a
  Stored in directory: /root/.cache/pip/wheels/96/b9/bb/994c1324b65e39dd1cd7b8ba92e5fb766dd77980929414a866
Successfully built gym
  Attempting uninstall: pygame
    Found existing installation: pygame 2.3.0
    Uninstalling pygame-2.3.0:
      Successfully uninstalled pygame-2.3.0
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys

sys.path.insert(0,'/content/drive/My Drive/ml-trde-notebooks')
data_path = '/content/drive/My Drive/ml-trde-notebooks/data'
model_path = "/content/drive/My Drive/ml-trde-notebooks/models"

Mounted at /content/drive


In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import reverb
import math

import tf_agents
from tf_agents.environments import suite_gym, tf_py_environment
from tf_agents.networks import network, utils, categorical_projection_network
from tf_agents.agents.ppo import ppo_agent
from tf_agents.policies import actor_policy
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.agents import tf_agent
from tf_agents.networks import actor_distribution_rnn_network
from tf_agents.networks import value_rnn_network
from tf_agents.utils import common
from tf_agents.environments import parallel_py_environment
from tf_agents.drivers import py_driver
from tf_agents.policies import py_tf_eager_policy
from tf_agents.replay_buffers import reverb_replay_buffer, tf_uniform_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.policies import policy_saver
from tensorflow import keras
import tensorflow_probability as tfp
from tf_agents.utils import common

from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import norm

import multiprocessing
from itertools import cycle

cores = multiprocessing.cpu_count()
print("I has cores: ", cores)
try:
  tf_agents.system.multiprocessing.enable_interactive_mode()
except Exception as inst:
  pass




I has cores:  12


In [4]:
# Executes this notebook in our space, making all of its functions/globals available
%run -i '/content/drive/My Drive/ml-trde-notebooks/backtest-common.ipynb'

Modified
1679922395
1679922395.0
1679922395.0


In [5]:
logging_level = 0
def error(*args):
    if logging_level >= 0:
        print(*args)

def info(*args):
    if logging_level >=1:
        print(*args)

def verbose(*args):
    if logging_level >=2:
        print(*args)

def debug(*args):
    if logging_level >=3:
        print(*args)

In [6]:
def download_stocks():
  # We don't want to train against crypto
  return get_all_stock_timerseries_for_csv("training_tickers3.csv", 3500, 1)

LENGTH_OF_STOCK_TRAINGING_DATA = 145 #I might need to fix this, but the model is tied to the number of symbols we trained on
def download_crypto():
  return get_all_product_timeseries(-1, 180, LENGTH_OF_STOCK_TRAINGING_DATA)

In [7]:
from_disk = False
use_crypto = False

if from_disk:
  product_data = load_dict(use_crypto)
  first_key = list(product_data.keys())[0]
  length = len(product_data[first_key])
elif use_crypto:
  product_data = download_crypto()
  save_dict(product_data, True)
else:
  product_data = download_stocks()
  save_dict(product_data)

min_len = min(len(df) for df in product_data.values())
print("symbols:", len(product_data.keys()))
print("min length:", min_len)
for name, df in product_data.items():
    product_data[name] = df.head(min_len)

product_data = dict(sorted(product_data.items(), reverse=True))

loading ticker:  XTSLA  count: 0 of: 164
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- XTSLA: No timezone found, symbol may be delisted
loading ticker:  XEL  count: 1 of: 164
[*********************100%***********************]  1 of 1 completed
loading ticker:  WU  count: 2 of: 164
symbols: 1
min length: 3500


In [8]:
import numpy as np
from gym import spaces
from tf_agents.environments import py_environment
from tf_agents.specs import array_spec
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step as ts

class TraderEnv(py_environment.PyEnvironment):
    
    def __init__(self, product, df):
        super(TraderEnv, self).__init__()

        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(4,), dtype=np.float64, minimum=np.float64(0), maximum=np.float64(np.inf), name='observation'
        )

        # Define action spec
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=2, name='action'
        )
        
        # Initialize environment state
        df = self.expand(df.copy())

        self.orig_timeseries = df
        self.timeseries = self.scale(df[["Date", "Close", "weighted-volume", "trend", "prob_above_trend"]])
        
        self._reset_vars()
        self.calculate_benchmark_metrics()


        # Don't add this here. Process the tickers into data files - you already have some.
        #self.lstm_internal = keras.models.load_model(model_path + "/lstm_cv-4.h15")
        #predictions = self.predict_trade(self.lstm_internal, self.timeseries[["Close", "weighted-volume"]])
        #self.timeseries["predicted-price"] = predictions
        
        self.product = product
        self.final = len(df)
    
    
    def calculate_benchmark_metrics(self):
      df = self.orig_timeseries
      row = df.iloc[0,:]
      price = self.get_price_with_slippage(row["Close"])
      self.benchmark_position_shares = math.floor(self.initial_cash/price)
     
    def predict_trade(self, model, X):
      predicted = model.predict(X, verbose=0).flatten()
     
      return predicted

    def expand(self, df):
      # Perform seasonal decomposition
      result = seasonal_decompose(df['Close'], model='additive', period=90, extrapolate_trend='freq')

      # Add trend back to original time series
      df["trend"] = result.trend

      # Compute the residuals by subtracting the trend from the original time series
      residuals = result.resid

      # Fit a Gaussian distribution to the residuals
      mu, std = norm.fit(residuals)

      # Compute the probability of a value being above or below the trend line
      # for each point in the time series
      z_scores = residuals / std
      df["prob_above_trend"] = 1 - norm.cdf(z_scores)
      df["weighted-volume"] = df["Close"] * df["Volume"]
      return df

    def scale(self, timeseries):
      df = timeseries.reset_index()  # Reset the index of the DataFrame
      dates = df['Date']
      data_to_scale = df.drop(['Date', 'index'], axis=1)
      self.scaler = MinMaxScaler()
      scaled_data = self.scaler.fit_transform(data_to_scale)
      scaled_df = pd.concat([dates, pd.DataFrame(scaled_data, columns=data_to_scale.columns)], axis=1)
      return scaled_df.set_index('Date')  # Set the index back to 'Date'

    def make_ledger_row(self):
        ledger = pd.DataFrame()
        ledger["Date"] = []
        ledger["Side"] = []
        ledger["Action"]  = []
        ledger["Price"] = []
        ledger["Fee"] = []
        return ledger

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset_vars(self):
        self._state = None
        self._episode_ended = False        
        self.ledger = self.make_ledger_row()
        self.slippage = .01
        self.fee = .0025
        self.current_index = 0
        self.cash = 5000
        self.initial_cash = self.cash
        self.position_shares = 0
        self.position_value = 0
        self.shares_owed = 0
        self.in_position = False
        self.in_long = False
        self.in_short = False
        self._state = self._get_initial_state()

    def _reset(self):
        # Reset the environment and return the initial time step
        self._reset_vars()
        return ts.restart(self._state)

    def _step(self, action):
        action = int(action)
        info("_step:", action)
        
        if self._episode_ended:
            # The last action ended the episode. Ignore the current action and start a new episode.
            return self.reset()

        # Apply the action and update the environment state
        self._apply_action(action)
        self._state = self._get_next_state()

        if self._is_episode_ended():
            self._episode_ended = True
            reward = self._get_final_reward()
            return ts.termination(self._state, reward)
        else:
            reward = self._get_reward()
            return ts.transition(self._state, reward=reward, discount=1.0)

    def _get_initial_state(self):
        # Return the initial state of the environment
        self.current_index = 0
        return self.env_block()

    def total_value(self):
      return self.position_value + self.cash

    def should_stop(self):
      # if cash is negative
      if (self.cash < 0):
        info("Bankrupt.")
        return True
      return False


    def _apply_action(self, action):
        # Advance the environment by one time step and return the observation, reward, and done flag
        self.current_index += 1
        info("step:", "index:", self.current_index, " of: ", self.final-1, "action: ", int(action))
        self.update_position_value()
        if (self.current_index >= self.final - 1 or  self.should_stop()):
          error("********MARKING DONE", "index:", self.current_index, " of: ", self.final-1, " cash: ", self.cash, " value: ", self.position_value)
          if (self.position_shares != 0):
            info("done so closing position")
            self.close_position()
          self._episode_ended = True
        else:
          self._episode_ended = False

        # AI says hold
        if (action == 0):
          info("holding action = 0.")
        # AI says long but we're already in a position
        elif (action == 1 and self.in_long):
          info("holding long.")
        # AI says long, we're not in a position, so buy
        elif (action == 1 and not self.in_position):
          info("opening long.")
          self.open_position()
        #AI says long, but we're short. Close the short, open a long.
        elif (action == 1 and self.in_short):
          info("closing short to open long.")
          self.close_short() 
          self.open_position()
        #AI says short, but we're already short  
        elif (action == 2 and self.in_short):
          info("holding short.")
        #AI says short and we're not in a position so exit 
        elif (action == 2 and not self.in_position):
          info("opening short.")
          self.open_short()
        #AI says short but we're long, close it
        elif (action == 2 and self.in_long):
          info("closing long to open short")
          self.close_position()
          info("opening short.")
          self.open_short()
        else: #assume hold
          info("unknown state! holding:", action, self.in_position, self.in_long, self.in_short)      
        self.update_position_value()
    
    def _get_next_state(self):
        # Calculate and return the next state based on the current state and action taken
        return self.env_block()

    def get_price_with_slippage(self, price):
        return price + (price * self.slippage)

    def open_position(self):
      self.in_position = True
      self.in_long = True
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      self.position_shares = math.floor(self.cash/price)
      self.cash = 0      
      ledger_row = self.make_ledger_row()
      ledger_row["Side"] = ["long"]
      ledger_row["Action"] = ["enter"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [self.fee]
      self.ledger = pd.concat([self.ledger, ledger_row])

    def open_short(self):
      self.in_position = True
      self.in_short = True
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      max_short_pos = math.floor(self.cash/price)
      self.shares_owed = max_short_pos
      self.cash = self.cash + (self.shares_owed * price)    
      info("Added cash on short: ", self.shares_owed * price, " total: ", self.cash, " took share debt:", self.shares_owed)  
      ledger_row = self.make_ledger_row()
      ledger_row["Side"] = ["short"]
      ledger_row["Action"] = ["enter"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [self.fee]
      self.ledger = pd.concat([self.ledger, ledger_row])

    def close_position(self):
      self.in_position = False
      self.in_long = False
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      value = price * self.position_shares
      self.position_shares = 0
      self.cash = self.cash + value
      ledger_row = self.make_ledger_row()
      ledger_row["Side"] = ["long"]
      ledger_row["Action"] = ["exit"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [self.cash * self.fee]
      self.ledger = pd.concat([self.ledger, ledger_row])

    def close_short(self):
      self.in_position = False
      self.in_short = False
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      value = price * self.shares_owed
      self.shares_owed = 0
      self.cash = self.cash - value
      ledger_row = self.make_ledger_row()
      ledger_row["Side"] = ["short"]
      ledger_row["Action"] = ["exit"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [self.cash * self.fee]
      self.ledger = pd.concat([self.ledger, ledger_row])


    def _is_episode_ended(self):
        return self._episode_ended
        
    def update_position_value(self):
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      self.position_value = (row["Close"] * self.position_shares) - (row["Close"] * self.shares_owed)
      self.benchmark_value = row["Close"] * self.benchmark_position_shares
      info("Calculating position value:",
            "current price:", row["Close"],
            "long shares:", self.position_shares,
            "short shares:", self.shares_owed,
            "long value:", row["Close"] * self.position_shares,
            "short debt:", row["Close"] * self.shares_owed,
            "position net value:", self.position_value)

    def _get_reward(self):
        self.update_position_value()
        current_portfolio_value = self.total_value()
        #compare to what we started with:
        #percentage_change = (current_portfolio_value - self.initial_cash) / self.initial_cash
        
        #compare to a bench mark
        percentage_change = ((current_portfolio_value - self.benchmark_value) / self.benchmark_value) * 100
        info("reward states: ",
              "position value: ", self.position_value,
              "shares: ", self.position_shares,
              "cash: ", self.cash,
              "benchmark value:", self.benchmark_value,
              "percentage_change: ", percentage_change)
        
        return percentage_change

    def _get_final_reward(self):
        # Calculate and return the final reward when the episode ends
        return self._get_reward()

    def env_block(self):
      start_index = self.current_index
      end_index = self.current_index
      df = self.timeseries.reset_index().drop(['Date'], axis=1)
      block = df.iloc[start_index].to_numpy()
      return block


In [12]:
import os
env_count = 0
num_parallel_envs = 4
learning_rate = 3e-4
num_epochs = 5 #25
discount_factor = 0.99
gradient_clipping = 0.5
entropy_regularization = 1e-2
value_pred_loss_coef = 0.5
use_gae = True
use_td_lambda_return = True
actor_loss_weight = 1.0
value_loss_weight = 0.5
log_interval = 5000
eval_interval = 100
train_model = False

def create_env():
    global env_count
    products = list(product_data.keys())
    product = products[env_count]
    env = TraderEnv(product, product_data[product])
    env_count+=1
    return env

def build_tf_uniform_replay_buffer(agent, batch_size):
    return tf_uniform_replay_buffer.TFUniformReplayBuffer(
        data_spec=agent.collect_data_spec,
        batch_size=batch_size,
        max_length=100000,
    )

def timestr():
  import datetime
  current_time = datetime.datetime.now()
  formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S") 
  return formatted_time

# Create the environment
all_envs = [create_env() for _ in range(len(product_data))]
sorted_envs = sorted(all_envs, key=lambda env: env.product)
all_envs = [tf_agents.environments.tf_py_environment.TFPyEnvironment(env) for env in sorted_envs]
#train_parallel_py_envs = [lambda: env for env in all_envs]
#train_parallel_env = parallel_py_environment.ParallelPyEnvironment(train_parallel_py_envs)
#train_env = tf_agents.environments.tf_py_environment.TFPyEnvironment(train_parallel_env)
#train_env = tf_agents.environments.tf_py_environment.TFPyEnvironment(create_env())



# Create the actor and value networks
# Create an ActorDistributionRnnNetwork with LSTM layers
actor_net = actor_distribution_rnn_network.ActorDistributionRnnNetwork(
    all_envs[0].observation_spec(),
    all_envs[0].action_spec(),
    input_fc_layer_params=None,
    lstm_size=(64,),  # LSTM layer with 64 units
    output_fc_layer_params=None,
)

# Create a ValueRnnNetwork with LSTM layers
value_net = value_rnn_network.ValueRnnNetwork(
    all_envs[0].observation_spec(),
    input_fc_layer_params=None,
    lstm_size=(64,),  # LSTM layer with 64 units
    output_fc_layer_params=None,
)


# Define optimizer, PPO hyperparameters, and create the PPO agent
#optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
global_step = tf.compat.v1.train.get_or_create_global_step()

clip_rho_threshold = 0.2
clip_pg_threshold = 0.2

tf_agent = tf_agents.agents.PPOAgent(
    all_envs[0].time_step_spec(),
    all_envs[0].action_spec(),
    optimizer,
    actor_net=actor_net,
    value_net=value_net,
    num_epochs=num_epochs,
    discount_factor=discount_factor,
    importance_ratio_clipping=clip_pg_threshold,
    entropy_regularization=entropy_regularization,
    gradient_clipping=gradient_clipping,
    value_pred_loss_coef=value_pred_loss_coef,
    use_gae=True,
    train_step_counter=global_step
)

# Initialize the policy
tf_agent.initialize()

eval_policy = tf_agent.policy
collect_policy = tf_agent.collect_policy

check_point_dir = os.path.join(model_path, "ppo-checkpoint")
train_checkpointer = common.Checkpointer(
    ckpt_dir=check_point_dir,
    agent=tf_agent,
    global_step=global_step)

policy_checkpointer = common.Checkpointer(
    ckpt_dir=os.path.join(check_point_dir, 'policy'),
    policy=eval_policy,
    global_step=global_step)
saved_model = policy_saver.PolicySaver(
    eval_policy, train_step=global_step)

save_model_path = os.path.join(model_path, "ppo-final")
train_checkpointer.initialize_or_restore()


if train_model:

  # Main training loop
  num_iterations = 4
  collect_episodes_per_iteration = 2

  tf_agent.train = common.function(tf_agent.train)
  
  train_env_count = 0
  for train_env in all_envs:
    #tf_agent.train_step_counter.assign(0)
    error(f"training {sorted_envs[train_env_count].product} count: {train_env_count} of {len(all_envs)}")
    replay_buffer = build_tf_uniform_replay_buffer(tf_agent, train_env.batch_size)
    rb_observer = replay_buffer.add_batch

    time_step = train_env.reset()
    print(time_step)
    collect_driver = py_driver.PyDriver(
        train_env,
        py_tf_eager_policy.PyTFEagerPolicy(
          tf_agent.collect_policy, use_tf_function=True),
        [rb_observer],
        max_steps=collect_episodes_per_iteration)

    dataset = replay_buffer.as_dataset(
        num_parallel_calls=3,
        sample_batch_size=64,
        num_steps=2).prefetch(3)

    iterator = iter(dataset)

    step = 0
    for _ in range(num_iterations):

      # Collect a few steps and save to the replay buffer.
      debug(f"running {step} at: ", timestr())
      time_step, _ = collect_driver.run(time_step)
      debug("step complete at: ", timestr())

      # Sample a batch of data from the buffer and update the agent's network.
      debug("training at:", timestr())
      experience, unused_info = next(iterator)
      train_loss = tf_agent.train(experience).loss
      debug("done training at:", timestr())

      step = tf_agent.train_step_counter.numpy()

      if step % log_interval == 0:
        error('step = {0}: loss = {1}'.format(step, train_loss))
        train_checkpointer.save(global_step=step)
        policy_checkpointer.save(global_step=step)
        saved_model.save(save_model_path)
    
else:
  tickerObj = yf.download(tickers = "CAT", interval = "1d")
  cat_df = pd.DataFrame(tickerObj)
  cat_df = cat_df.reset_index()

  env = TraderEnv("CAT", cat_df)
  eval_env = tf_py_environment.TFPyEnvironment(env)
  saved_policy = tf.keras.models.load_model(save_model_path)

  def run_episodes(policy, eval_tf_env):
    num_episodes = 3
    frames = []
    for _ in range(num_episodes):
      time_step = eval_tf_env.reset()
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = eval_tf_env.step(action_step.action)
  run_episodes(saved_policy, env)

[*********************100%***********************]  1 of 1 completed

ValueError: ignored

In [ ]:
policy = tf_agent.policy
tf.saved_model.save(policy, model_path + "/ppo-final")